In [1]:
import torch
import torchvision
from collections import namedtuple
import os
import matplotlib.pyplot as plt




In [2]:
from attacks.analytic_attack import ImprintAttacker
from modifications.imprint import ImprintBlock
from utils.breaching_utils import *

# Attack begins here:

### Initialize your model

In [3]:
setup = dict(device=torch.device("cpu"), dtype=torch.float)

# This could be any model:
model = torchvision.models.resnet18()
model.eval()
loss_fn = torch.nn.CrossEntropyLoss()
# It will be modified maliciously:
input_dim = data_cfg_default.shape[0] * data_cfg_default.shape[1] * data_cfg_default.shape[2]
num_bins = 100 # Here we define number of imprint bins
block = ImprintBlock(input_dim, num_bins=num_bins)
model = torch.nn.Sequential(
    torch.nn.Flatten(), block, torch.nn.Unflatten(dim=1, unflattened_size=data_cfg_default.shape), model
)
secret = dict(weight_idx=0, bias_idx=1, shape=tuple(data_cfg_default.shape), structure=block.structure)
secrets = {"ImprintBlock": secret}

### And your dataset (ImageNet by default)

In [4]:
# transforms = torchvision.transforms.Compose(
#     [
#         torchvision.transforms.Resize(256),
#         torchvision.transforms.CenterCrop(224),
#         torchvision.transforms.ToTensor(),
#         torchvision.transforms.Normalize(mean=data_cfg_default.mean, std=data_cfg_default.std),
#     ]
# )
# dataset = torchvision.datasets.ImageNet(root="~/data/imagenet/", split="val", transform=transforms)
# batch_size = 64 # Number of images in the user's batch. We have a small one here for visualization purposes
# import random
# random.seed(123) # You can change this to get a new batch. 
# samples = [dataset[i] for i in random.sample(range(len(dataset)), batch_size)]
# data = torch.stack([sample[0] for sample in samples])
# labels = torch.tensor([sample[1] for sample in samples])

In [5]:
import medmnist
from medmnist import INFO, Evaluator

batch_size = 8 # Number of images in the user's batch. We have a small one here for visualization purposes
import random
random.seed(234324) # You can change this to get a new batch.

transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=data_cfg_default.mean, std=data_cfg_default.std),
    ]
)


data_flag = 'dermamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])
dataset = DataClass(split="val", transform=transforms, download=True, size=224)
samples = [dataset[i] for i in random.sample(range(len(dataset)), batch_size)]
data = torch.stack([sample[0] for sample in samples])
labels = torch.tensor([sample[1] for sample in samples]).flatten()

Using downloaded and verified file: /Users/maximilianeckert/.medmnist/dermamnist_224.npz


/var/folders/c4/ywdtx99d1vl0ptsg1fy494_40000gn/T/ipykernel_1772/2130135357.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  labels = torch.tensor([sample[1] for sample in samples]).flatten()


In [6]:
print(labels)

tensor([5, 5, 5, 5, 0, 5, 5, 5])


### Simulate an attacked FL protocol

In [7]:
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from torch.utils.data import DataLoader

model = ModuleValidator.fix(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Fix: Added missing code for optimizer initialization
training_set = DataClass(split="train", transform=transforms, download=True, size=224)
training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)

#if hasattr(model, "autograd_grad_sample_hooks"):
#    del model.autograd_grad_sample_hooks

privacy_engine = PrivacyEngine()
model, optimizer, data_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=training_loader, 
    noise_multiplier=1.1,
    max_grad_norm=1.0,
)
# Run a forward pass to ensure activations are detected
data, labels = next(iter(training_loader))

# Ensure labels are 1D
if labels.ndim > 1:
    labels = labels.flatten()

# Forward pass to ensure activations are detected
_ = model(data)

Using downloaded and verified file: /Users/maximilianeckert/.medmnist/dermamnist_224.npz


/opt/homebrew/lib/python3.11/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [8]:
print(model)

GradSampleModule(Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): ImprintBlock(
    (linear0): Linear(in_features=150528, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=150528, bias=True)
    (nonlin): ReLU()
  )
  (2): Unflatten(dim=1, unflattened_size=(3, 224, 224))
  (3): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): GroupNorm(32, 64, eps=1e-05, affine=True)
      )
      (1): B

In [9]:

# This is the attacker:
attacker = ImprintAttacker(model, loss_fn, attack_cfg_default, setup)

# Server-side computation:
queries = [dict(parameters=[p for p in model.parameters()], buffers=[b for b in model.buffers()])]
server_payload = dict(queries=queries, data=data_cfg_default)
# User-side computation:
loss = loss_fn(model(data), labels)
shared_data = dict(
    gradients=[torch.autograd.grad(loss, model.parameters())],
    buffers=None,
    num_data_points=1,
    labels=labels,
    local_hyperparams=None,
)

ValueError: No activations detected for <class 'torch.nn.modules.linear.Linear'>, run forward after add_hooks(model)

### Reconstruct data from the update

In [ ]:
# Attack:
reconstructed_user_data, stats = attacker.reconstruct(server_payload, shared_data, secrets, dryrun=False)

In [ ]:
# Metrics?: 
from utils.analysis import report
true_user_data = {'data': data, 'labels': labels}
metrics = report(reconstructed_user_data,
    true_user_data,
    server_payload,
    model, compute_ssim=False) # Can change to true and install a package...
print(f"MSE: {metrics['mse']}, PSNR: {metrics['psnr']}, LPIPS: {metrics['lpips']}, SSIM: {metrics['ssim']} ")

### Plot ground-truth data

In [ ]:
plot_data(data_cfg_default, true_user_data, setup)

# Create the "images" folder if it doesn't exist
if not os.path.exists("images"):
    os.makedirs("images")

# Save the images inside the "images" folder
plt.savefig("images/true_user_data.png")


### Now plot reconstructed data

In [ ]:
plot_data(data_cfg_default, reconstructed_user_data, setup)
# Save the images inside the "images" folder
plt.savefig("images/reconstructed_user_data.png")